In [1]:
using PCT

Precompiling PCT
  ✓ PCT
  1 dependency successfully precompiled in 2 seconds. 7 already precompiled.


## $x^{\dagger} A x$

In [2]:
f, _ = @pct (A::Her) -> pullback((x::CV) -> sum((i, j), x(i)' * A(i, j) * x(j)))
blaserize(f) |> latex |> println

A \to \mathcal{P}\left(x \to x^{\dagger}\cdot A\cdot x\right)


In [3]:
df = redux(vdiff(f), settings=symmetry_settings) |> blaserize |> latex |> println

A \to x \to 2.0 \cdot A\cdot x


## Trace of Product

In [4]:
f, _ = @pct begin
    tr = (X::CM) -> sum(i, X(i, i))
    matmul = (A::CM, B::CM) -> (i::N, j::N) -> sum(n, A(i, n) * B(n, j))
    pullback((A::CM, B::CM) -> tr(matmul(A, B) + matmul(B, A)))
end; f

let 
    tr = (X) -> 
        ∑((i), X(i, i))
    matmul = (A, B) -> 
        (i, j) -> 
            ∑((n), (A(i, n)⋅B(n, j)))
    𝒫((A, B) -> 
        tr((matmul(A, B)+matmul(B, A))))
end

In [5]:
eval_all(f)

𝒫((A, B) -> 
    ∑((i), (∑((n), (A(n, i)⋅B(i, n)))+∑((n), (A(i, n)⋅B(n, i))))))

In [8]:
redux(vdiff(eval_all(f)); settings=symmetry_settings)  |> blaserize |> latex |> println

\left(A, B\right) \to \left(2.0 \cdot B^{\dagger}, 2.0 \cdot A^{\dagger}\right)


## Conjugate Gradient

In [10]:
f, _ = @pct (A::Sym, r::RV, p::RV, b::RV, x::RV) -> begin
    E = (x::RV) -> sum((i, j), 0.5 * x(i) * A(i, j) * x(j)) + sum(i, x(i) * b(i))
    pullback((α::R, β::R) -> E((i::N) -> x(i) + α * (r(i) + β * p(i))))
end; blaserize(f)

(A, r, p, b, x) -> 
    let 
        E = ((x) -> 
            xᵀ⋅b+(x) -> 
            (0.5⋅xᵀ⋅A⋅x))
        𝒫((α, β) -> 
            E((x+α⋅(r+β⋅p))))
    end

In [13]:
df = vdiff(f) |> blaserize |> latex |> println

\left(A, r, p, b, x\right) \to \mathrm{let}\\ \quad E = \left(x \to x^{T}\cdot b+x \to 0.5\cdot x^{T}\cdot A\cdot x\right)\\\quad \left(α, β\right) \to \left(\nabla \left(E\right)\left(\left(x+α \cdot \left(r+β \cdot p\right)\right)\right)^{T}\cdot \left(r+β \cdot p\right), \nabla \left(E\right)\left(\left(x+α \cdot \left(r+β \cdot p\right)\right)\right)^{T}\cdot p\cdot α\right)\\ \mathrm{end}


In [12]:
df = redux(vdiff(eval_all(f)); settings=symmetry_settings) |> blaserize |> latex |> println

\left(A, r, p, b, x\right) \to \left(α, β\right) \to \left(\left(\left(r+β \cdot p\right)^{T}\cdot b+\left(x+α \cdot \left(r+β \cdot p\right)\right)^{T}\cdot A\cdot \left(r+β \cdot p\right)\right), α\cdot \left(b^{T}\cdot p+\left(x+α \cdot \left(r+β \cdot p\right)\right)^{T}\cdot A\cdot p\right)\right)


## Lagrangian Mechanics

In [53]:
_, ctx = @pct begin
    @space X begin
        type = (RF, ) -> RF
        linear = true
    end
end; 

In [54]:
f, _ = @pct _ ctx (L::RO, D::X) -> pullback((r::RF) -> sum((t::R), L(r(t), D(r)(t)))); f

(L, D) -> 
    𝒫((r) -> 
        ∑((t), L(r(t), D(r)(t))))

In [55]:
redux(vdiff(f); settings=custom_settings(:clench_sum=>true))

(L, D) -> 
    (r) -> 
        ((_d) -> 
            𝒫((_z_2) -> 
                L(_z_2, D(r)(_d)))(r(_d), 1)+D'((_d) -> 
            𝒫((_z_1) -> 
                L(r(_d), _z_1))(D(r)(_d), 1)))

## Hartree Fock Gradient and Hessian Vector Product

In [14]:
_, ctx = @pct begin    
    @space T begin
        type = (I, I, I, I) -> C
        symmetries = (((2, 1, 4, 3), :conj), ((3, 4, 1, 2), :id))
    end
end;

In [15]:
f, _ = @pct _ ctx (J::T) -> pullback((C::CM) -> sum((i, j, p, q, r, s),
    C(p, i)' * C(q, i) * C(r, j)' * C(s, j) * J(p, q, r, s))); f

(J) -> 
    𝒫((C) -> 
        ∑((i, j, p, q, r, s), (C(p, i)'⋅C(r, j)'⋅J(p, q, r, s)⋅C(s, j)⋅C(q, i))))

In [16]:
df = redux(vdiff(f); settings=symmetry_settings)

(J) -> 
    (C) -> 
        (_d, _d_1) -> 
            (4.0⋅∑((i, p, q, r), (C(q, i)'⋅J(_d, r, q, p)⋅C(p, i)⋅C(r, _d_1))))

In [17]:
df |> get_body |> decompose |> pp |> simplify |> first

(C, _k) -> 
    (_a, _a_1) -> 
        (∑((_d, i, p, q), (J(_d, _a, q, p)'⋅C(p, i)'⋅4.0⋅_k(_d, _a_1)⋅C(q, i)))+∑((_d, _d_1, q, r), (J(_d, r, q, _a)'⋅C(r, _d_1)'⋅4.0⋅_k(_d, _d_1)⋅C(q, _a_1)))+∑((_d, _d_1, p, r), (4.0⋅J(_d, r, _a, p)⋅_k(_d, _d_1)⋅C(p, _a_1)⋅C(r, _d_1))))

## Foster Boys

In [25]:
f, _ = @pct (R::Her, R2::Her) -> pullback((C::CM) -> begin 
        r = (i::N) -> sum((p, q), R(p, q) * C(p, i)' * C(q, i))
        sum(i, sum((p, q), R2(p, q) * C(p, i)' * C(q, i))) - sum(i, r(i)' * r(i))
    end); blaserize(f)

(R, R2) -> 
    𝒫((C) -> 
        let 
            r = (i) -> 
                (C⁺⋅R⋅C)(i, i)
            (tr(C⋅C⁺⋅R2)(-1.0⋅r⁺⋅r))
        end)

In [27]:
df = redux(vdiff(eval_all(f)); settings=symmetry_settings) |> blaserize

(R, R2) -> 
    (C) -> 
        (-4.0⋅((_d, _d_1) -> 
            ((R⋅C)(_d, _d_1)⋅(C⁺⋅R⋅C)(_d_1, _d_1)))+2.0⋅R2⋅C)

## Localized Wannier Functions

In [20]:
_, ctx = @pct begin
    @domain BZ begin
        base = I
        periodic = true
    end
    
    @domain X begin
        symmetric = true
        contractable = false
    end
    
    @space Mmn begin
        type = (N, N, BZ, BZ) -> C
        symmetries = (((2, 1, 4, 3), :conj),)
    end

    @space SV begin
        type = (I,) -> C
        symmetries = (((1,), :ineg),)
    end
    
    @space Gauge begin
        type = (N, N, BZ) -> C
    end
end;

In [21]:
f, _ = @pct _ ctx (S::Mmn, w::SV) -> pullback((U::Gauge) -> 
    begin
        ρ = (n::N, b::X) -> sum((k::BZ, p, q), U(p, n, k)' * S(p, q, k, k + b) * U(q, n, k + b))
        sum((n, b::X), ρ(n, b)' * ρ(n, b))
    end); f

(S, w) -> 
    𝒫((U) -> 
        let 
            ρ = (n, b) -> 
                ∑((k, p, q), (U(p, n, k)'⋅U(q, n, (b+k))⋅S(p, q, k, (b+k))))
            ∑((n, b), (ρ(n, b)'⋅ρ(n, b)))
        end)

In [22]:
# df = vdiff(eval_all(f); settings=custom_settings(:logging=>true; preset = symmetry_settings))
df = redux(vdiff(eval_all(f)); settings=symmetry_settings)

(S, w) -> 
    (U) -> 
        (_d, _d_1, _d_2) -> 
            (4.0⋅∑((b, p, k, _p, q), (U(_p, _d_1, (b+k))'⋅U(q, _d_1, k)⋅U(p, _d_1, (_d_2+b))⋅S(_d, p, _d_2, (_d_2+b))⋅S(_p, q, (b+k), k))))

In [23]:
_, ctx = @pct begin
    @space Tau begin
        type=(R, RV) -> RM
    end
    
    @space Nu begin
        type=(RV, ) -> R
    end
        
    @space S begin
        type=(RV, )->RV
    end
end;

In [66]:
f, _ = @pct _ ctx (τ::Tau, ν::Nu, p::S) -> :vdiff(pullback((x::RV) -> τ(ν(x), p(x)))); f

(τ, ν, p) -> 
    vdiff̂(𝒫((x) -> 
        τ(ν(x), p(x))))

In [15]:
df = process_directive(f)

(τ, ν, p) -> 
    (x) -> 
        (𝒫(ν)(x, 𝒫((_z) -> 
            τ(_z, p(x)))(ν(x), 1))+𝒫(p)(x, 𝒫((_z_1) -> 
            τ(ν(x), _z_1))(p(x), 1)))